In [1]:
import warnings
warnings.filterwarnings('ignore')
sys.path.insert(1, '../..')

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
# "" = CPU
# "0" = GPU_0
# "1" = GPU_1
os.environ["CUDA_VISIBLE_DEVICES"]="0"; 
# print( 'CUDA Version =', os.environ['CUDA_VERSION'] )

import sys
print( 'Python Version =', sys.version.split()[0] )

import tensorflow as tf
print( 'Tensorflow Version =',tf.__version__)

print("GPU Available: ", tf.test.is_gpu_available())

NameError: name 'sys' is not defined

In [2]:
import os

from os.path import join
from utils_1 import *
import shutil
import gc
from inspect import getsource
import time

# import wandb
# from wandb.tensorflow import WandbHook
# from wandb.keras import WandbCallback

# wandb.init(project="woundsee", sync_tensorboard=True)

ModuleNotFoundError: No module named 'utils_1'

In [3]:
def load_x( paths ):
    def get_x_rgb( path ):
        rgb = read_image( path, 'rgb' )
        return rgb / 255

    get_x = get_x_rgb
    x = np.array( [ get_x(path) for path in paths ] ).astype( np.float32 )
    return x

def load_y( paths ):
    def get_y( path ):
        grayscale = read_image( path, 'gray' ) / 63
        layers = [  (grayscale==i) + [0] for i in range(0, 5) ]
        
        return np.stack( layers, axis=2 )

    y = np.array( [ get_y(path) for path in paths ] ).astype( np.float32 )
    return y

In [4]:
def get_generator( this_fold_dir, t, color ):
    def my_generator( samples, batch_size, color ):
        L = len( samples )
        while True:
            batch_start = 0
            batch_end = batch_size
            shuffle(samples)
            f_paths = [ os.path.join( feature_dir, i ) for i in samples ]
            l_paths = [ os.path.join( label_dir, i ) for i in samples ]
            while batch_start < L:
                limit = min( batch_end, L )
                x = load_x( f_paths[batch_start:limit] )
                y = load_y( l_paths[batch_start:limit] )
                yield x, y
                batch_start += batch_size
                batch_end += batch_size
    
#     if not (t == "Train" or t == "Validate"):
#         raise ValueError('t must be Train or Validate')
    
    my_dir = os.path.join( this_fold_dir, t )
    feature_dir = os.path.join( my_dir, 'Features' )
    label_dir = os.path.join( my_dir, 'Labels' )
    
    
    samples = [ p for p in os.listdir( feature_dir ) if 'checkpoint' not in p]
    
    if isTest:
        samples = samples[:256]
    
    genarator = my_generator( samples, batch_size, color )
    return genarator, len(samples)

In [5]:
def train( fold ):
    
    this_ex_dir = os.path.join( this_experiment_dir, str(fold) )
    
    if not isTest:
        os.mkdir( this_ex_dir )
    
    this_fold_dir = os.path.join( dataset, str(fold) )
    
    print( "Start Process fold: %d" % (fold) )
    train_gen, num_train_samples = get_generator( this_fold_dir, "Train", 'rgb' )
    val_gen, num_val_samples = get_generator( this_fold_dir, "Validate", 'rgb' )
#     val_gen, num_val_samples = get_generator( '/notebooks/VOLUME_1TB/Thesis_dataset', "Test_Sub-Images", 'rgb' )
    
    print( "Train: %d, Val: %d" % ( num_train_samples, num_val_samples ) )

    
    callbacks = get_call_backs(this_ex_dir)
    model     = get_model( input_height, input_width, input_chanel )
    
    if isTest:
        model.summary()
    
    train_history = model.fit_generator( 
        generator = train_gen
        ,steps_per_epoch = np.ceil( num_train_samples / batch_size )
        ,validation_data = val_gen
        ,validation_steps = np.ceil( num_val_samples / batch_size )
        ,epochs=epochs
        ,callbacks=callbacks
    )
    
    if not isTest:
        np.save( os.path.join( this_ex_dir, 'history') , np.array(train_history.history))
    del model

In [6]:
def get_tensorboard_callback( this_ex_dir ):
    from tensorflow.keras.callbacks import TensorBoard
    path = os.path.join( os.path.join(this_ex_dir, 'tensorboard') )
    os.mkdir(path)
    print( path )
    tbCallBack = TensorBoard(log_dir=path,profile_batch=0, histogram_freq=0, write_graph=True, write_images=True)
    return tbCallBack

In [7]:
def get_adaptive_lr_callback():
    from tensorflow.keras.callbacks import LearningRateScheduler    
    adaptive_lr_callback=LearningRateScheduler(lr_scheduler)
    return adaptive_lr_callback

In [8]:
def get_call_backs(this_ex_dir):
    
    adaptive_lr = get_adaptive_lr_callback()
    callbacks = [ adaptive_lr ]
    
    if not isTest:
        log_cb = get_log_callback(this_ex_dir)
        callbacks.append( log_cb )
        
        tensorboard = get_tensorboard_callback( this_ex_dir )
        callbacks.append( tensorboard )
        
#         callbacks.append(WandbCallback())
        
    return callbacks

In [9]:
def setup_experiment():
    def create_experiment_dir():
        os.mkdir( this_experiment_dir )
        print(this_experiment_dir)
        
    def create_detail_of_experiment():
        detail_path = os.path.join( this_experiment_dir, 'detail.txt' )
        with open( detail_path, 'w', encoding="utf-8" ) as file:
            pass
        return detail_path
    
    def write_line_to_detail( msg ):
        with open( detail_path, 'a', encoding="utf-8" ) as file:
            file.write( "%s\n\n" % msg )
            
    def write_model_to_detail( model ):
        with open( detail_path, 'a', encoding="utf-8" ) as file:
            model.summary(print_fn=lambda x: file.write(x + '\n'))
            
    model = get_model( input_height, input_width, input_chanel )
    
    create_experiment_dir()
    detail_path = create_detail_of_experiment()
    
    write_line_to_detail( 'Method: %s' % method )
    write_line_to_detail( 'Structure: %s' % structure )
    write_line_to_detail( 'Input Shape: %s' % str( ( input_height, input_width, input_chanel) ) )
    write_line_to_detail( 'Color: %s' % color )
    write_line_to_detail( 'Description: %s' % desc )
    write_line_to_detail( 'Batch size: %d' % batch_size )
    write_line_to_detail( 'Epochs: %d' % epochs )
    write_line_to_detail( 'Start lr: %s' % learning_rate)
    
    adaptive = getsource( lr_scheduler )
    write_line_to_detail( adaptive )
    
    loss_func = getsource( loss )
    write_line_to_detail( loss_func )
    
    write_model_to_detail( model )

In [10]:
def get_model( input_height, input_width, input_chanel ):
    
    _dict = {
        'u-net-res-2': get_model_unet_res_2,
    }
    
    if structure not in _dict.keys():
        raise ValueError('%s not in our model.' % structure)
    
    return _dict[structure]( input_height, input_width, input_chanel )

In [11]:
def start( start, end  ):
    
    def start_train():
        for i in range(start,end):
            print("_" * 100)
            gc.collect()
            train(i)
            print("_" * 100)
        gc.collect()
    
    def calculate_environment_name():
        global this_experiment_dir
        number_folder       = len(os.listdir(Experiments))
        this_experiment_dir = os.path.join( Experiments, str( number_folder ).zfill(4) )
        print( "Experiments: %s" % str( number_folder ).zfill(4) )
        
    calculate_environment_name()
    
    if isTest:
        start_train()
        return
    
    setup_experiment()
    start_train()

In [12]:
def get_log_callback(this_ex_dir, log_name = 'log.txt', prefix_model=''):
    log_path = os.path.join( this_ex_dir, log_name)
    from tensorflow.keras.callbacks import Callback
    from tensorflow.keras import backend as K
    
    class My_Callback_1(Callback):
        def __init__(self, filename, separator=' ; '):
            self.sep = separator
            self.filename = filename
            self.val_a = -1
            self.best_epoch = -1
            super(My_Callback_1, self).__init__()
        
        def on_epoch_begin(self, epoch, logs=None):
            self.tic = time.time()
        
        def on_epoch_end(self, epoch, logs=None):
            toc = time.time()
            t_sec = round( toc - self.tic)
            
            lr_var = self.model.optimizer.lr
            lr = K.get_value(lr_var)
            
            logs = logs or {}
            val = [ "%4d" % epoch, "%7d" % t_sec, "%10s" % lr ]
            for key in sorted( logs.keys() ):
                if key == 'lr':
                    continue
                value = logs[key]
                val.append( "%3.6f" % value )
            line = self.sep.join( val )
            with open( self.filename, 'a' ) as f:
                f.write( "\n%s" % line )
            
            if epoch+1 >= 50 and (epoch+1) % 10 == 0:
                model_name = "%s%d_model.h5" % (prefix_model,epoch+1)
                model_path = os.path.join( this_ex_dir, model_name )
                self.model.save( model_path )
                
                
            val_n = logs[ "val_dice" ]
            if val_n >= self.val_a:
                self.val_a = val_n
                self.best_epoch = epoch
                model_name = "%sbest_model.h5" % (prefix_model)
                model_path = os.path.join( this_ex_dir, model_name )
                self.model.save( model_path )
                with open( self.filename, 'a' ) as f:
                    f.write( " ***" )
                
                
        def on_train_begin(self, logs=None):
            line = "epochs" + self.sep + "secs" + self.sep + "lr" + self.sep + self.sep.join( sorted(self.params['metrics']) )
            with open( self.filename, 'w' ) as f:
                f.write( "%s" % line )
        
        def on_train_end(self, logs=None):
            model_name = "%send_model.h5" % prefix_model
            model_path = os.path.join( this_ex_dir, model_name )
            self.model.save( model_path )
            with open( self.filename, 'a' ) as f:
                f.write( "\n\nBest Epoch: %d, val__dice: %.6f" % (self.best_epoch,self.val_a) )
    
    my_callback = My_Callback_1(log_path)
    
    return my_callback

In [13]:
def get_model_unet_res_2( input_height, input_width, input_chanel ):
    from tensorflow.keras.layers import ZeroPadding2D, Input, Conv2D, MaxPooling2D, SpatialDropout2D, concatenate, Conv2DTranspose, BatchNormalization, Activation, Add
    from tensorflow.keras.models import Model
    from tensorflow.keras.optimizers import Adam
    from tensorflow.keras.activations import relu, softmax
    
    def residual_block( filters, kernel_size ):
        def block( X ):
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X_shortcut = X
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = BatchNormalization() ( X )
            X = Activation( relu ) (X)
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Add()( [ X_shortcut, X ] )
            X = BatchNormalization() ( X )
            X = Activation( relu ) (X)
            X_shortcut = X
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = BatchNormalization() ( X )
            X = Activation( relu ) (X)
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Add()( [ X_shortcut, X ] )
            X = BatchNormalization() ( X )
            X = Activation( relu ) (X)
            return X
        return block
    
    def residual_block_2( filters, kernel_size ):
        def block( X ):
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X_shortcut = X
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Activation( relu ) (X)
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Add()( [ X_shortcut, X ] )
            X = Activation( relu ) (X)
            X_shortcut = X
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Activation( relu ) (X)
            X = Conv2D( filters=filters, kernel_size=kernel_size, padding='same' ) ( X )
            X = Add()( [ X_shortcut, X ] )
            X = Activation( relu ) (X)
            return X
            
        return block
    
    # INPUT
    input_shape = ( input_height, input_width, input_chanel )
    input_      = Input( input_shape )
    
    X = Conv2D( filters=16, kernel_size=(7,7), padding='same' ) ( input_ )
    
    encode_1 = residual_block( filters=32, kernel_size=(3, 3) ) ( X )
    pool_1 = MaxPooling2D( pool_size=(2,2) ) ( encode_1 )
    
    encode_2 = residual_block( filters=64, kernel_size=(3, 3) ) ( pool_1 )
    pool_2 = MaxPooling2D( pool_size=(2,2) ) ( encode_2 )
    
    encode_3 = residual_block( filters=128, kernel_size=(3, 3) ) ( pool_2 )
    pool_3 = MaxPooling2D( pool_size=(2,2) ) ( encode_3 )
    
    mid_1 = residual_block( filters=256, kernel_size=(3, 3) ) (pool_3)
    
    decode_3 = Conv2DTranspose( filters=128, kernel_size=(3, 3), strides=(2, 2), padding='same') (mid_1)
    decode_3 = concatenate([ encode_3, decode_3 ], axis=-1)
    decode_3 = residual_block_2( filters=128, kernel_size=(3, 3) ) ( decode_3 )
    decode_3 = SpatialDropout2D( 0.1 ) (decode_3)
    
    decode_2 = Conv2DTranspose( filters=64, kernel_size=(3, 3), strides=(2, 2), padding='same') (decode_3)
    decode_2 = concatenate([ encode_2, decode_2 ], axis=-1)
    decode_2 = residual_block_2( filters=64, kernel_size=(3, 3) ) ( decode_2 )
    decode_2 = SpatialDropout2D( 0.1 ) (decode_2)
    
    decode_1 = Conv2DTranspose( filters=32, kernel_size=(3, 3), strides=(2, 2), padding='same') (decode_2)
    decode_1 = concatenate([ encode_1, decode_1 ], axis=-1)
    decode_1 = residual_block_2( filters=32, kernel_size=(3, 3) ) ( decode_1 )
    
    output = SpatialDropout2D( 0.2 ) ( decode_1 )
    output = Conv2D(filters=5, kernel_size=(1, 1), padding='same' ) ( output )
    output = Activation( softmax ) (output)
    
    model = Model( inputs=[input_], outputs=[output], name='U-Net with ResNet Block' )
    optimizer = Adam( lr=learning_rate )
    model.compile( optimizer=optimizer, loss=loss, metrics=[dice, iou, cate] )
    
    return model

In [14]:
def iou(y_true, y_pred):
    from tensorflow.keras import backend as K
    smooth = 1
    y_pred = y_pred[...,1:]
    y_true = y_true[...,1:]
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
    return ( intersection + smooth ) / ( union + smooth )
    

def dice(y_true, y_pred ):
    from tensorflow.keras import backend as K
    smooth = 1
    y_pred = y_pred[...,1:]
    y_true = y_true[...,1:]
    intersection = K.sum(y_true * y_pred, axis=[1,2,3])
    denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
    return (2. * intersection + smooth) / (denom + smooth)
    

def cate(y_true, y_pred ):
    from tensorflow.keras.metrics import categorical_accuracy
    return categorical_accuracy(y_true, y_pred)

In [15]:
def loss(y_true, y_pred):
    from tensorflow import keras
    from tensorflow.keras import backend as K
    
    def dice(y_true, y_pred):
        y_pred = y_pred[...,1:]
        y_true = y_true[...,1:]
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        denom = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3])
        d = (2. * intersection + smooth) / (denom + smooth)
        return d
    
    def iou(y_true, y_pred):
        intersection = K.sum(y_true * y_pred, axis=[1,2,3])
        union = K.sum(y_true, axis=[1,2,3]) + K.sum(y_pred, axis=[1,2,3]) - intersection
        return ( intersection + smooth ) / ( union + smooth )
    
    smooth = 1
    d = K.pow(1-dice(y_true, y_pred), 2)
    return d

In [16]:
def lr_scheduler(epoch, lr):
        decay_rate = .5
        decay_step = 25
        if epoch % decay_step == 0 and epoch >= 50:
            return lr * decay_rate
        return lr

In [19]:
method        = 'Sub-images'
isTest        = False
epochs        = 200
batch_size    = 4
learning_rate = 1e-04
color         = 'rgb'
structure     = 'u-net-res-2'

height = 512
width  = 512
# stride = 128

input_height, input_width, input_chanel = ( height, width, 3 )

dataset_name        = "wound_rajavithi"
k_fold              = 10

root                = join(os.getcwd(), "data", dataset_name, "wound_segmentation")

source_dir          = join(root, "training_k_fold")

Experiments         = join(root, "experiments")

if not is_exists( Experiments ):
    os.mkdir( Experiments )

In [20]:
for p in [
    source_dir
]:
    global dataset, desc, this_experiment_dir 
    desc    = p
    dataset = p
    
    print(p)
    start( 1, k_fold+1)

/home/deploy/Documents/woundsee-ai/jupyter/data/wound_rajavithi/wound_segmentation/training_k_fold
Experiments: 0036
/home/deploy/Documents/woundsee-ai/jupyter/data/wound_rajavithi/wound_segmentation/experiments/0036
____________________________________________________________________________________________________
Start Process fold: 1
Train: 72, Val: 8
/home/deploy/Documents/woundsee-ai/jupyter/data/wound_rajavithi/wound_segmentation/experiments/0036/1/tensorboard


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Wandb version 0.8.27 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/200
18/18 [==============================] - 23s 1s/step - loss: 0.7798 - dice: 0.1192 - iou: 0.0646 - cate: 0.1960 - val_loss: 0.8466 - val_dice: 0.0812 - val_iou: 0.0430 - val_cate: 0.0780
Epoch 2/200
18/18 [==============================] - 8s 456ms/step - loss: 0.6884 - dice: 0.1749 - iou: 0.0984 - cate: 0.2912 - val_loss: 0.8830 - val_dice: 0.0607 - val_iou: 0.0315 - val_cate: 0.0314
Epoch 3/200
18/18 [==============================] - 9s 475ms/step - loss: 0.5646 - dice: 0.2621 - iou: 0.1588 - cate: 0.3317 - val_loss: 0.8431 - val_dice: 0.0835 - val_iou: 0.0445 - val_cate: 0.1737
Epoch 4/200
18/18 [==============================] - 9s 491ms/step - loss: 0.4952 - dice: 0.3191 - iou: 0.2040 - cate: 0.5176 - val_loss: 0.8299 - val_dice: 0.0900 - val_iou: 0.0476 - val_cate: 0.7188
Epoch 5/200
18/18 [==============================] - 8s 429ms/step - loss: 0.4391 - dice: 0.3747 - iou: 0.2534 - cate: 0.6718 - val_loss: 0.9015 - val_dice: 0.0519 - val_iou: 0.0274 - val_cate: 0.72

Epoch 52/200
18/18 [==============================] - 8s 435ms/step - loss: 0.2315 - dice: 0.5527 - iou: 0.4023 - cate: 0.7999 - val_loss: 0.6348 - val_dice: 0.2227 - val_iou: 0.1379 - val_cate: 0.8511
Epoch 53/200
18/18 [==============================] - 8s 429ms/step - loss: 0.2170 - dice: 0.5613 - iou: 0.4062 - cate: 0.8115 - val_loss: 0.3908 - val_dice: 0.4003 - val_iou: 0.2646 - val_cate: 0.8487
Epoch 54/200
18/18 [==============================] - 8s 432ms/step - loss: 0.2261 - dice: 0.5610 - iou: 0.4111 - cate: 0.8091 - val_loss: 0.4390 - val_dice: 0.3636 - val_iou: 0.2377 - val_cate: 0.8595
Epoch 55/200
18/18 [==============================] - 8s 448ms/step - loss: 0.2006 - dice: 0.5836 - iou: 0.4302 - cate: 0.8201 - val_loss: 0.3649 - val_dice: 0.4244 - val_iou: 0.2859 - val_cate: 0.8711
Epoch 56/200
18/18 [==============================] - 8s 463ms/step - loss: 0.2183 - dice: 0.5622 - iou: 0.4082 - cate: 0.8108 - val_loss: 0.4047 - val_dice: 0.4098 - val_iou: 0.2841 - val_cat

Epoch 103/200
18/18 [==============================] - 8s 465ms/step - loss: 0.1088 - dice: 0.6934 - iou: 0.5426 - cate: 0.8653 - val_loss: 0.4462 - val_dice: 0.3840 - val_iou: 0.2687 - val_cate: 0.8762
Epoch 104/200
18/18 [==============================] - 9s 497ms/step - loss: 0.1129 - dice: 0.6862 - iou: 0.5336 - cate: 0.8645 - val_loss: 0.5488 - val_dice: 0.3042 - val_iou: 0.2071 - val_cate: 0.8659
Epoch 105/200
18/18 [==============================] - 8s 459ms/step - loss: 0.0958 - dice: 0.7157 - iou: 0.5695 - cate: 0.8737 - val_loss: 0.4318 - val_dice: 0.4009 - val_iou: 0.2852 - val_cate: 0.8907
Epoch 106/200
18/18 [==============================] - 8s 448ms/step - loss: 0.0991 - dice: 0.7079 - iou: 0.5590 - cate: 0.8710 - val_loss: 0.3457 - val_dice: 0.4595 - val_iou: 0.3246 - val_cate: 0.8776
Epoch 107/200
18/18 [==============================] - 8s 433ms/step - loss: 0.0963 - dice: 0.7108 - iou: 0.5616 - cate: 0.8745 - val_loss: 0.4618 - val_dice: 0.3729 - val_iou: 0.2604 - va

...

Epoch 200/200
18/18 [==============================] - 8s 469ms/step - loss: 0.1921 - dice: 0.6006 - iou: 0.4516 - cate: 0.8285 - val_loss: 0.3706 - val_dice: 0.4299 - val_iou: 0.2974 - val_cate: 0.8012
____________________________________________________________________________________________________
____________________________________________________________________________________________________
Start Process fold: 10
Train: 72, Val: 8
/home/deploy/Documents/woundsee-ai/jupyter/data/wound_rajavithi/wound_segmentation/experiments/0036/10/tensorboard
Epoch 1/200
18/18 [==============================] - 60s 3s/step - loss: 0.7826 - dice: 0.1201 - iou: 0.0666 - cate: 0.1034 - val_loss: 0.7412 - val_dice: 0.1414 - val_iou: 0.0774 - val_cate: 0.1147
Epoch 2/200
18/18 [==============================] - 9s 506ms/step - loss: 0.6793 - dice: 0.1868 - iou: 0.1096 - cate: 0.1493 - val_loss: 0.7207 - val_dice: 0.1540 - val_iou: 0.0851 - val_cate: 0.1941
Epoch 3/200
18/18 [====================

18/18 [==============================] - 8s 438ms/step - loss: 0.3088 - dice: 0.4900 - iou: 0.3528 - cate: 0.7846 - val_loss: 0.6439 - val_dice: 0.2214 - val_iou: 0.1387 - val_cate: 0.4337
Epoch 50/200
18/18 [==============================] - 8s 469ms/step - loss: 0.3342 - dice: 0.4560 - iou: 0.3167 - cate: 0.7766 - val_loss: 0.4885 - val_dice: 0.3258 - val_iou: 0.2094 - val_cate: 0.5900
Epoch 51/200
18/18 [==============================] - 8s 436ms/step - loss: 0.3124 - dice: 0.4817 - iou: 0.3423 - cate: 0.7782 - val_loss: 0.5903 - val_dice: 0.2527 - val_iou: 0.1570 - val_cate: 0.4216
Epoch 52/200
18/18 [==============================] - 8s 430ms/step - loss: 0.3035 - dice: 0.4860 - iou: 0.3436 - cate: 0.7934 - val_loss: 0.4872 - val_dice: 0.3198 - val_iou: 0.2004 - val_cate: 0.5980
Epoch 53/200
18/18 [==============================] - 8s 427ms/step - loss: 0.3006 - dice: 0.4933 - iou: 0.3529 - cate: 0.7940 - val_loss: 0.4766 - val_dice: 0.3334 - val_iou: 0.2147 - val_cate: 0.6833
Epo

Epoch 75/200
18/18 [==============================] - 8s 470ms/step - loss: 0.3133 - dice: 0.4756 - iou: 0.3339 - cate: 0.7904 - val_loss: 0.4065 - val_dice: 0.4167 - val_iou: 0.2982 - val_cate: 0.6567
Epoch 76/200
18/18 [==============================] - 8s 431ms/step - loss: 0.2872 - dice: 0.4980 - iou: 0.3520 - cate: 0.7839 - val_loss: 0.3968 - val_dice: 0.3934 - val_iou: 0.2595 - val_cate: 0.6654
Epoch 77/200
18/18 [==============================] - 8s 438ms/step - loss: 0.2766 - dice: 0.5087 - iou: 0.3618 - cate: 0.7945 - val_loss: 0.3002 - val_dice: 0.4765 - val_iou: 0.3286 - val_cate: 0.6651
Epoch 78/200
18/18 [==============================] - 8s 432ms/step - loss: 0.2596 - dice: 0.5277 - iou: 0.3804 - cate: 0.8082 - val_loss: 0.3602 - val_dice: 0.4393 - val_iou: 0.3077 - val_cate: 0.6809
Epoch 79/200
18/18 [==============================] - 8s 430ms/step - loss: 0.2703 - dice: 0.5141 - iou: 0.3661 - cate: 0.8072 - val_loss: 0.4242 - val_dice: 0.3998 - val_iou: 0.2838 - val_cat

Epoch 126/200
18/18 [==============================] - 8s 451ms/step - loss: 0.1628 - dice: 0.6218 - iou: 0.4656 - cate: 0.8533 - val_loss: 0.3255 - val_dice: 0.4629 - val_iou: 0.3206 - val_cate: 0.6900
Epoch 127/200
18/18 [==============================] - 8s 427ms/step - loss: 0.1600 - dice: 0.6237 - iou: 0.4663 - cate: 0.8522 - val_loss: 0.3207 - val_dice: 0.4668 - val_iou: 0.3233 - val_cate: 0.6891
Epoch 128/200
18/18 [==============================] - 8s 432ms/step - loss: 0.1743 - dice: 0.6038 - iou: 0.4447 - cate: 0.8451 - val_loss: 0.3156 - val_dice: 0.4690 - val_iou: 0.3238 - val_cate: 0.6913
Epoch 129/200
18/18 [==============================] - 8s 434ms/step - loss: 0.1539 - dice: 0.6320 - iou: 0.4752 - cate: 0.8514 - val_loss: 0.3089 - val_dice: 0.4755 - val_iou: 0.3295 - val_cate: 0.6923
Epoch 130/200
18/18 [==============================] - 8s 472ms/step - loss: 0.1646 - dice: 0.6164 - iou: 0.4581 - cate: 0.8470 - val_loss: 0.2944 - val_dice: 0.4897 - val_iou: 0.3418 - va

Epoch 177/200
18/18 [==============================] - 8s 438ms/step - loss: 0.1385 - dice: 0.6500 - iou: 0.4935 - cate: 0.8594 - val_loss: 0.3109 - val_dice: 0.4728 - val_iou: 0.3258 - val_cate: 0.6914
Epoch 178/200
18/18 [==============================] - 8s 457ms/step - loss: 0.1318 - dice: 0.6566 - iou: 0.4996 - cate: 0.8636 - val_loss: 0.3099 - val_dice: 0.4708 - val_iou: 0.3221 - val_cate: 0.6902
Epoch 179/200
18/18 [==============================] - 8s 427ms/step - loss: 0.1405 - dice: 0.6503 - iou: 0.4961 - cate: 0.8611 - val_loss: 0.3114 - val_dice: 0.4730 - val_iou: 0.3264 - val_cate: 0.6918
Epoch 180/200
18/18 [==============================] - 9s 482ms/step - loss: 0.1503 - dice: 0.6376 - iou: 0.4822 - cate: 0.8584 - val_loss: 0.2812 - val_dice: 0.4740 - val_iou: 0.3132 - val_cate: 0.6486
Epoch 181/200
18/18 [==============================] - 8s 435ms/step - loss: 0.1370 - dice: 0.6525 - iou: 0.4971 - cate: 0.8623 - val_loss: 0.3116 - val_dice: 0.4733 - val_iou: 0.3270 - va